In [1]:
import numpy as np
import pandas as pd
import folium, requests
from folium.plugins import HeatMap
import requests,json,os
from urllib.parse import quote 

In [2]:
df = pd.read_csv('data/main/lines/1호선.csv')
df.head()

사용월  호선명     지하철역  출근시간 하차인원  09-16시 승차인원  09-16시 하차인원  퇴근시간 승차인원  \
0  201501  1호선       가능      23287        96744        66805      31775   
1  201501  1호선  가산디지털단지     230457       107638       189559     209852   
2  201501  1호선       간석      15467        78718        55206      28721   
3  201501  1호선       개봉      52104       280814       191890      90855   
4  201501  1호선       관악      22169       100507        83845      40644   

   퇴근시간 하차인원  새벽 승차인원  새벽 하차인원  야간 승차인원  야간 하차인원  총 승차인원  총 하차인원  
0      60948     8225     1240    17566    55647  190620  146979  
1      86319     8884     1544   114317    58461  274951  480021  
2      55488     5554     1282    17595    56137  164392  128092  
3     217153    15614     3593    50829   184806  565027  432393  
4      65829     6103     1307    19637    61074  210027  168395

In [3]:
df_st = pd.read_csv('data/st_addr_20230529.csv')
df_a = df[['호선명','지하철역']].copy()
df_a.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df_a.reset_index(drop=True,inplace=True)
df_st.rename(columns={'역명':'지하철역'},inplace=True)
df_st.head()

지하철역                       도로명주소
0    서울  서울특별시 중구 세종대로 지하2(남대문로 5가)
1    시청     서울특별시 중구 세종대로 지하101(정동)
2    종각     서울특별시 종로구 종로 지하55(종로1가)
3  종로3가    서울특별시 종로구 종로 지하129(종로3가)
4  종로5가    서울특별시 종로구 종로 지하216(종로5가)

In [4]:
res = pd.merge(df_a,df_st,on='지하철역',how='left')
res.tail()

호선명   지하철역                  도로명주소
122  1호선     회룡  경기도 의정부시 평화로 363(회룡역)
123  1호선  동두천중앙      경기도 동두천시 동두천로 228
124  1호선     쌍용  충청남도 천안시 서북구 쌍용19로 20
125  1호선   평택지제       경기도 평택시 경기대로 777
126  1호선     탕정   충남 아산시 탕정면 한들물빛6로 11

In [5]:
def kakao_location(place):
    with open('../DataAnalysis/04.지도시각화/data/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [6]:
def rtn_addr(target):
    str_addr = df_st[df_st.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [ ]:
temp1 =[]
for i in res.index:
    try:
        target = res['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, res.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
res = pd.concat([res, df_test], axis=1)

In [ ]:
res.to_csv('latlng_test.csv',index=False)

In [7]:
res = pd.read_csv('latlng_test.csv')
res.drop(['도로명주소','호선명'],inplace=True, axis=1)
result = pd.merge(df,res,on='지하철역',how='left')
result.head()

사용월  호선명     지하철역  출근시간 하차인원  09-16시 승차인원  09-16시 하차인원  퇴근시간 승차인원  \
0  201501  1호선       가능      23287        96744        66805      31775   
1  201501  1호선  가산디지털단지     230457       107638       189559     209852   
2  201501  1호선  가산디지털단지     230457       107638       189559     209852   
3  201501  1호선       간석      15467        78718        55206      28721   
4  201501  1호선       개봉      52104       280814       191890      90855   

   퇴근시간 하차인원  새벽 승차인원  새벽 하차인원  야간 승차인원  야간 하차인원  총 승차인원  총 하차인원        lat  \
0      60948     8225     1240    17566    55647  190620  146979  37.748369   
1      86319     8884     1544   114317    58461  274951  480021  37.481596   
2      86319     8884     1544   114317    58461  274951  480021  37.481596   
3      55488     5554     1282    17595    56137  164392  128092  37.464688   
4     217153    15614     3593    50829   184806  565027  432393  37.494522   

          lng  
0  127.044221  
1  126.882559  
2  126.882559  
3  126.693451  
4  126.859103

In [8]:
result.to_csv('latlng_merge_test.csv',index=False)